In [1]:
import os
from datetime import datetime
import requests
import tiktoken
import json
from typing import Dict, Tuple
from dotenv import load_dotenv
load_dotenv()

model = "gpt-4o-mini"
encoding = tiktoken.encoding_for_model(model)

In [79]:
headers = {
    'Content-Type': 'application/json'
}
freshdesk_api_key = os.environ.get('FRESHDESK_API_KEY')
auth = (freshdesk_api_key, 'X')
domain = 'nueca'

import requests

tickets_url = f"https://{domain}.freshdesk.com/api/v2/tickets?order_by=created_at&order_type=desc&per_page=3"
ticket_response = requests.get(tickets_url, headers=headers, auth=auth)
tickets = ticket_response.json() if ticket_response.status_code == 200 else []

In [80]:
def parse_freshdesk_conversations(conversations, ticket_description) -> dict:
    # sorted_conversations = sorted(conversations, key=lambda x: x['created_at'])
    
    customer_conversation = ""
    agent_conversation = ""

    message_data = {
        'id': ticket_description['id'],
        'body_text': ticket_description['subject'],
        'role': 'Customer'
    }

    customer_conversation += message_data['body_text'] + "\n"

    for i, conv in enumerate(conversations, 2):
        message_data = {
            'body_text': conv['body_text'],
        }
        if message_data['body_text'] == '':
            continue
        if conv['incoming']:
            customer_conversation += message_data['body_text'] + "\n"
        else:
            agent_conversation += message_data['body_text'] + "\n"

    return {
        'customer_conversation': customer_conversation,
        'agent_conversation': agent_conversation,
    }

In [81]:
def get_all_conversations(ticket_id, ticket_description):
    conversations = []
    url = f"https://{domain}.freshdesk.com/api/v2/tickets/{ticket_id}/conversations"

    while url:
        response = requests.get(url, headers=headers, auth=auth)
        if response.status_code != 200:
            break
        data = response.json()
        conversations.extend(data)
        link = response.headers.get('link')
        if link and 'rel="next"' in link:
            import re
            match = re.search(r'<([^>]+)>;\s*rel="next"', link)
            url = match.group(1) if match else None
            print("has pagination")
        else:
            url = None

    # print(conversations)
    parsed_conversations = parse_freshdesk_conversations(conversations, ticket_description)

    return parsed_conversations

In [82]:
def add_to_map(mmap: Dict[int, Tuple[int, int]], key: int, value: Tuple[int, int]):
    if key in mmap:
        existing = mmap[key]
        mmap[key] = (existing[0] + value[0], existing[1] + value[1])
    else:
        mmap[key] = value

    return mmap

In [83]:
mmap: Dict[int, Tuple[int, int]] = {}
mmap

{}

In [ ]:
# 11810

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

months = set(())

for ticket_id in range(7713, 22606):
    description_url = f"https://{domain}.freshdesk.com/api/v2/tickets/{ticket_id}"
    ticket_description = requests.get(description_url, headers=headers, auth=auth).json()
    
    date_str = ticket_description['created_at']
    dt = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
    month = dt.month

    parsed_conversations = get_all_conversations(ticket_id, ticket_description)
    
    customer_messages = parsed_conversations['customer_conversation']
    customer_length = len(customer_messages)
    
    agent_messages = parsed_conversations['agent_conversation']
    agent_length = len(agent_messages)

    mmap = add_to_map(mmap, month, (customer_length, agent_length))

    if ticket_id % 100 == 0:
        print(f'ID: {ticket_id} - {mmap}')
        print("-----")

-----

#### Monthly conversation length

In [134]:
mmap = {1: (403353, 669959), 2: (356369, 684548), 3: (418523, 822324)}

In [135]:
tickets = 13796 - 7713
tickets

6083

#### Monthly number of tickets

In [136]:
total_tickets_month = tickets / 3
round(total_tickets_month)

2028

#### Average conversation length per ticket

In [137]:
total_length = 0

for key in mmap:
    customer_length = mmap[key][0]
    agent_length = mmap[key][1]
    total_length += customer_length + agent_length

ave_conversation_len = total_length / total_tickets_month
round(ave_conversation_len)

1655

### Computation per ticket

In [ ]:
# gpt-4o-mini
model = "gpt-4o-mini"

# [$5 • $15]
model = "chatgpt-4o-latest" 

tokens:

vector population (20,000 tickets)
    Input
        - get problem and resolution prompt no conversation [4405 len] (990 tokens)
        - ave conversation length [2612 len] (584 tokens)
    Output
        - problem and resolution [617 len] (105 tokens)

answering tickets
rag classification
    Input
        - get problem prompt no context [5019 len] (1023 tokens) 
            - query issues [171 len]
            - specific_issues [3271 len]
            - problem [257 len]
        - top 5 relevant tickets (context) [2560 len] (453 tokens)
    Output
        - query and specific issue (10 tokens)

rag
    Input
        - resolution prompt no context [3035 len] (604 tokens) 
            - problem [257 len]
        - top 5 relevant tickets (context) [2560 len] (453 tokens)
    Output
        - answer [366 len] (88 tokens)
        - priority (5 tokens)

### API pricing

In [138]:
mini_input_pricing = 0.15
mini_output_pricing = 0.6

flagship_input_pricing = 5
flagship_output_pricing = 15

dollar_php_rate = 56.62

### Vector DB population price (20,000 tickets)

In [139]:
total_tickets = 20000

In [140]:
get_problem_and_resolution_prompt = 990
ave_conversation_length = 584

problem_and_resolution = 105

In [141]:
input_tokens = get_problem_and_resolution_prompt + ave_conversation_length
output_tokens = problem_and_resolution

In [142]:
total_input_tokens = input_tokens * total_tickets
total_output_tokens = output_tokens * total_tickets

In [143]:
mini_input_price = (total_input_tokens / 1e6) * mini_input_pricing
mini_output_price = (total_output_tokens / 1e6) * mini_output_pricing

mini_input_price_peso = mini_input_price * dollar_php_rate
mini_output_price_peso = mini_output_price * dollar_php_rate

In [144]:
flagship_input_price = (total_input_tokens / 1e6) * flagship_input_pricing
flagship_output_price = (total_output_tokens / 1e6) * flagship_output_pricing

flagship_input_price_peso = flagship_input_price * dollar_php_rate
flagship_output_price_peso = flagship_output_price * dollar_php_rate

In [145]:
print("COST OF VECTOR POPULATION")

print(f"Total input tokens: {total_input_tokens:,}")
print(f"Total output tokens: {total_output_tokens:,}")
print()

print(f"Dollar to Peso: {dollar_php_rate}")
print()

print(f'GPT-4o mini (Cheap) [I/O $0.15 • $0.6]')
print(f"Mini input pricing: ${round(mini_input_price, 2)} or ₱{round(mini_input_price_peso, 2)}")
print(f"Mini output pricing: ${round(mini_output_price, 2)} or ₱{round(mini_output_price_peso, 2)}")
print()

print(f'ChatGPT-4o latest (Flagship) [I/O $5 • $15]')
print(f"Flagship input pricing: ${round(flagship_input_price, 2)} or ₱{round(flagship_input_price_peso, 2)}")
print(f"Flagship output pricing: ${round(flagship_output_price, 2)} or ₱{round(flagship_output_price_peso, 2)}")

COST OF VECTOR POPULATION
Total input tokens: 31,480,000
Total output tokens: 2,100,000

Dollar to Peso: 56.62

GPT-4o mini (Cheap) [I/O $0.15 • $0.6]
Mini input pricing: $4.72 or ₱267.36
Mini output pricing: $1.26 or ₱71.34

ChatGPT-4o latest (Flagship) [I/O $5 • $15]
Flagship input pricing: $157.4 or ₱8911.99
Flagship output pricing: $31.5 or ₱1783.53


### Price of answering one ticket

In [159]:
problem_prompt = 1023
problem_context = 453
answer_prompt = 604
answer_context = 453

core_problem = 53
query_and_specific_issue = 10
answer = 88
priority = 5

In [160]:
total_input_tokens = problem_prompt + problem_context + answer_prompt + answer_context
total_output_tokens = core_problem + query_and_specific_issue + answer + priority

In [161]:
mini_input_price = (total_input_tokens / 1e6) * mini_input_pricing
mini_output_price = (total_output_tokens / 1e6) * mini_output_pricing

mini_input_price_peso = mini_input_price * dollar_php_rate
mini_output_price_peso = mini_output_price * dollar_php_rate

In [162]:
flagship_input_price = (total_input_tokens / 1e6) * flagship_input_pricing
flagship_output_price = (total_output_tokens / 1e6) * flagship_output_pricing

flagship_input_price_peso = flagship_input_price * dollar_php_rate
flagship_output_price_peso = flagship_output_price * dollar_php_rate

In [168]:
print("COST OF ANSWERING ONE TICKET")

print(f"Total input tokens: {total_input_tokens:,}")
print(f"Total output tokens: {total_output_tokens:,}")
print()

print(f"Dollar to Peso: {dollar_php_rate}")
print()

print(f'GPT-4o mini (Cheap) [I/O $0.15 • $0.6]')
print(f"Mini input pricing: ${mini_input_price} or ₱{round(mini_input_price_peso, 2)}")
print(f"Mini output pricing: ${mini_output_price} or ₱{round(mini_output_price_peso, 2)}")
print()

print(f'ChatGPT-4o latest (Flagship) [I/O $5 • $15]')
print(f"Flagship input pricing: ${round(flagship_input_price, 3)} or ₱{round(flagship_input_price_peso, 2)}")
print(f"Flagship output pricing: ${flagship_output_price} or ₱{round(flagship_output_price_peso, 2)}")

COST OF ANSWERING ONE TICKET
Total input tokens: 2,533
Total output tokens: 156

Dollar to Peso: 56.62

GPT-4o mini (Cheap) [I/O $0.15 • $0.6]
Mini input pricing: $0.00037995 or ₱0.02
Mini output pricing: $9.36e-05 or ₱0.01

ChatGPT-4o latest (Flagship) [I/O $5 • $15]
Flagship input pricing: $0.013 or ₱0.72
Flagship output pricing: $0.00234 or ₱0.13


### Price of answering tickets per month

In [151]:
total_tickets_month

2027.6666666666667

In [152]:
problem_prompt = 1023
problem_context = 453
answer_prompt = 604
answer_context = 453

core_problem = 53
query_and_specific_issue = 10
answer = 88
priority = 5

In [153]:
total_input_tokens = problem_prompt + problem_context + answer_prompt + answer_context
total_output_tokens = core_problem + query_and_specific_issue + answer + priority

In [154]:
montly_input_tokens = total_input_tokens * total_tickets_month
montly_output_tokens = total_output_tokens * total_tickets_month

In [155]:
mini_input_price = (montly_input_tokens / 1e6) * mini_input_pricing
mini_output_price = (montly_output_tokens / 1e6) * mini_output_pricing

mini_input_price_peso = mini_input_price * dollar_php_rate
mini_output_price_peso = mini_output_price * dollar_php_rate

In [156]:
flagship_input_price = (montly_input_tokens / 1e6) * flagship_input_pricing
flagship_output_price = (montly_output_tokens / 1e6) * flagship_output_pricing

flagship_input_price_peso = flagship_input_price * dollar_php_rate
flagship_output_price_peso = flagship_output_price * dollar_php_rate

In [157]:
print("MONTHLY COST OF ANSWERING TICKETS")

print(f"Monthly tickets: {round(total_tickets_month, 2):,}")
print()

print(f"Total input tokens: {round(montly_input_tokens, 2):,}")
print(f"Total output tokens: {round(montly_output_tokens, 2):,}")
print()

print(f"Dollar to Peso: {dollar_php_rate}")
print()

print(f'GPT-4o mini (Cheap) [I/O $0.15 • $0.6]')
print(f"Mini input pricing: ${round(mini_input_price, 2):,} or ₱{round(mini_input_price_peso, 2):,}")
print(f"Mini output pricing: ${round(mini_output_price, 2):,} or ₱{round(mini_output_price_peso, 2):,}")
print()

print(f'ChatGPT-4o latest (Flagship) [I/O $5 • $15]')
print(f"Flagship input pricing: ${round(flagship_input_price, 2):,} or ₱{round(flagship_input_price_peso, 2):,}")
print(f"Flagship output pricing: ${round(flagship_output_price, 2):,} or ₱{round(flagship_output_price_peso, 2):,}")

MONTHLY COST OF ANSWERING TICKETS
Monthly tickets: 2,027.67

Total input tokens: 5,136,079.67
Total output tokens: 316,316.0

Dollar to Peso: 56.62

GPT-4o mini (Cheap) [I/O $0.15 • $0.6]
Mini input pricing: $0.77 or ₱43.62
Mini output pricing: $0.19 or ₱10.75

ChatGPT-4o latest (Flagship) [I/O $5 • $15]
Flagship input pricing: $25.68 or ₱1,454.02
Flagship output pricing: $4.74 or ₱268.65


-----

In [80]:
constant = 754 / 3373
constant

0.223539875481767

In [ ]:
prompt_template = """
## ROLE & CONTEXT
You are a customer support representative for Tindahang Tapat, a digital platform enabling sari-sari stores to order groceries via mobile phone. Generate appropriate responses using only the provided knowledge base context.

## PRIMARY OBJECTIVE
Deliver accurate, helpful solutions to customer problems based strictly on available context information, while maintaining professional service standards.

## RESPONSE METHODOLOGY

### Content Requirements
- **Context fidelity**: Use ONLY information from the provided context
- **Solution focus**: Prioritize actionable steps and clear guidance
- **Completeness**: Address all aspects of the customer's problem when context allows
- **Accuracy**: Never infer or assume details not explicitly stated in context

### Communication Standards
- **Professional tone**: Courteous, confident, and solution-oriented
- **Filipino market awareness**: Use terminology familiar to sari-sari store owners
- **Clarity**: Avoid technical jargon; use simple, direct language
- **Brevity**: Concise responses (2-4 sentences) that fully address the issue

### Fallback Protocol
When context is insufficient, use exactly: *"I don't have specific information about this issue in my current resources. Please contact our support team at [support contact] for immediate assistance with your concern."*

## INPUT DATA

### Knowledge Base Context
```
{context}
```

### Customer Problem
```
{problem}
```

## RESPONSE GENERATION RULES

### Content Validation
1. **Context check**: Ensure solution exists in provided context
2. **Completeness check**: Verify all problem aspects are addressed
3. **Accuracy check**: Confirm no assumptions beyond context are made
4. **Tone check**: Maintain professional, helpful customer service voice

### Quality Standards
- **Actionable**: Include specific steps when solutions are available
- **Comprehensive**: Address root cause when context provides sufficient detail
- **Preventive**: Mention prevention tips if included in context
- **Follow-up ready**: Set clear expectations for next steps if needed

## OUTPUT REQUIREMENTS
With no additional commentary, explanations, or formatting, always return output in the following two sections:

ANSWER:  
[Provide a complete customer response based on context - 2-4 sentences addressing their problem with specific, actionable guidance in Taglish. If the context does not include relevant information, use exactly: "ANSWER: I don't have specific information about this issue in my current resources. Please contact our support team at [support contact] for immediate assistance with your concern."]

PRIORITY:  
[Provide one of the following values only: Low, Medium, High, Urgent, or NONE. If the fallback answer was used, return PRIORITY as NONE.]
"""

prompt = prompt_template.format(context="", problem=problem)
prompt_len = len(prompt)

model = "gpt-4o-mini"
encoding = tiktoken.encoding_for_model(model)
mini_tokens = len(encoding.encode(prompt))

model = "chatgpt-4o-latest"
encoding = tiktoken.encoding_for_model(model)
big_tokens = len(encoding.encode(prompt))

print(f'Prompt length: {prompt_len}')
print(f'GPT-4o mini tokens: {mini_tokens}')
print(f'ChatGPT-4o latest tokens: {big_tokens}')

In [103]:
problem = ""
resolution = "Para makapagbago ng delivery address sa iyong order, siguraduhing nasa tamang bahagi ka ng order placement process kung saan makikita ang option para sa delivery details. Kung hindi mo pa rin mabago ang address, subukang ulitin ang proseso o i-refresh ang app. Kung magpapatuloy ang problema, puwede kang makipag-ugnayan sa aming support team para sa agarang tulong."

model = "gpt-4o-mini"
encoding = tiktoken.encoding_for_model(model)
mini_tokens = len(encoding.encode(problem))
mini_tokens += len(encoding.encode(resolution))

model = "chatgpt-4o-latest"
encoding = tiktoken.encoding_for_model(model)
big_tokens = len(encoding.encode(problem))
big_tokens += len(encoding.encode(resolution))

print(f'Prompt length: {len(problem) + len(resolution)}')
print(f'GPT-4o mini tokens: {mini_tokens}')
print(f'ChatGPT-4o latest tokens: {big_tokens}')

Prompt length: 366
GPT-4o mini tokens: 88
ChatGPT-4o latest tokens: 88


In [36]:
query_type_choices = ["Product & Delivery Issues","Product & Delivery Inquiries & Request","Loyalty & Promotions","APP Issues & Inquiries","Miscellaneous Queries","Store Account & Profile Management", "NONE"]

query_issues = ""

for choice in query_type_choices:
    query_issues += choice + "\n"

len(query_issues)

171

In [37]:
specific_issue_choices = ["Additional order","Advance delivery concern","Assumed incorrect sukli","Assumed missing items","Bank transfer inquiry","B.O. guidelines inquiry","Cancellation of order due to delay","Cancellation of order/items due to store situation","Cancellation of double order","Cancellation of order due to no cash","Cancellation due to closed store upon delivery","Check payment inquiry","Complaint of ordered OS product","Confirmation if ordered items are in stock","Appreciation of good service","Confirmation if the order has been received","Container deposit inquiry","Credit card inquiry","Delivery inquiry","Delivery reminders","Different amount in app vs. received","Discount inquiry","Entire order unserved","Excess items not included in SI","Factory defect","Faulty products","First-time customer inquiry","Follow-up on an order not placed","Follow-up on delayed delivery","Follow-up on unresolved concern","Forgot/wrong box offloaded","Gcash inquiry","Gifts/Freebies inquiry","Holiday schedule inquiry","How to join inquiry","Incorrect packaging","Incorrect sukli","Inquiry of actual vs. ordered amount","Location suggestion/inquiry","Minimum order inquiry","Missing items included in SI","Order pick-up request/inquiry","Order via mobile/FB","Ordered OS promo items","Others","Packaging reminders","Pahabol na order after cut-off","Price change inquiry","Product inquiry/suggestions","Received near expiry item","Relay message to AG","Request to reschedule delivery","Requested a specific time for delivery","Request to deliver 2 orders together","Request to deliver to another address","Request to re-deliver due to closed store","Request to replace wrong ordered SKU","Request to separate SI of two orders","Store accessories inquiry","Stock availability inquiry","VAT/Receipt confusion","Warehouse walk-in request","Wrong variant received","KPM inquiry","Total points inquiry","Total ticket inquiry","Content quantity per box inquiry","Request to cancel if undelivered","Request for advance delivery","Paid excess amount","Claiming of prize/voucher","How to redeem points","To redeem points","CRM entry","Wrong receipt received","Receipt doubled","Follow-up on prize","Application crashes","Application inquiry","Assumed REORDER button as EDIT for a sent order","Cannot enter the app due to slow connection","Cannot open the application","Cannot order through application","Cannot update app due to country settings","Deactivate account","Distorted image","Google Play concerns","High minimum quantity per order","History deletion","How to edit sent order","How to log in","How to log in on multiple devices","Huawei issue","iOS inquiry","Navigation issues","Network error","No delivery schedule in the app","Platform usage","Points suggestion","Price/unit is incorrect","Recovery of account","Re-download of application","Request a feature to edit sent order","Slow internet connection","SSL handshake issue","UI/UX concerns","Updating errors","Wrong delivery schedule in app","Wrong product photo","Wrong price","Wrong quantity","Ask to be verified after re-signing up","Assist in signing up","Cancellation of registration","Edit of user credentials","Follow-up visit to add address","Follow-up verification","Follow-up visit to change address","Forgotten password/Change of password","Request to change address","Request to change number","Request to edit store name","Request to recover account","Problem with delivery team","Availed credit","Closed app inquiry","Requesting for donations","Received wrong order", "NONE"]

specific_issues = ""

for choice in specific_issue_choices:
    specific_issues += choice + "\n"

len(specific_issues)

3271

In [38]:
problem = "Problem: Customer experienced difficulty with delivery address and subsequent order placement due to application unavailability. The customer initially requested to change their delivery address. The application became unresponsive after the address change."
len(problem)

257

In [85]:
# Failure ticket
ticket_id

13990

'created_at': '2025-07-03T00:55:24Z'


tickets_url = f"https://nueca.freshdesk.com/api/v2/tickets/7133"
ticket_description = requests.get(tickets_url, headers=headers, auth=auth).json()

date_str = ticket_description['created_at']
dt = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
month = dt.month
month

date_str